In [32]:
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
import pickle


In [5]:
grade = "3"
target_path = f"../../data/question_correctness/.csv"
scores_path = "../../Gates.ReadComp_By-Item_Gr3-5(CM).xlsx"
corpus_path = f"../../subtest_txt/gr{grade}_paragraphs.txt"
questions_path = f"../../subtest_txt/gr{grade}_questions.txt"

In [4]:
sub_tests = []
with open(corpus_path,'r') as fp:
    sub_test = fp.readline()
    while sub_test:
        sub_tests.append(sub_test)
        sub_test = fp.readline()


df = pd.read_excel(scores_path)
sub_test_number = df.columns[1:]

print(sub_test_number)

questions_ranges = [(1,5), (6,8), (9,13), (14,16), (17,21), (22,27), (28,30),
                   (31,35), (36,40), (41,43), (44,48)]

model = SentenceTransformer('paraphrase-mpnet-base-v2')

#Change the length to 200
model.max_seq_length = 500

sub_tests_embed = model.encode(sub_tests, show_progress_bar=True)

print(sub_tests_embed.shape)

Index(['Gr3.RC.Gates_01', 'Gr3.RC.Gates_02', 'Gr3.RC.Gates_03',
       'Gr3.RC.Gates_04', 'Gr3.RC.Gates_05', 'Gr3.RC.Gates_06',
       'Gr3.RC.Gates_07', 'Gr3.RC.Gates_08', 'Gr3.RC.Gates_09',
       'Gr3.RC.Gates_10',
       ...
       'Gr5.RC.Gates_45', 'Gr5.RC.Gates_46', 'Gr5.RC.Gates_47',
       'Gr5.RC.Gates_48', 'Gr5.RC.Gates_RawScore', 'Gr5.RC.Gates_GradeEquiv',
       'Gr5.RC.Gates_NCE', 'Gr5.RC.Gates_NPR', 'Gr5.RC.Gates_NS',
       'Gr5.RC.Gates_ESS'],
      dtype='object', length=162)



(12, 768)


In [18]:
questions = []
answers = []

with open(questions_path,'r') as fp:
    for _ in range(48):
        question = fp.readline()
        answer = fp.readline()
#         print(question,answer)
        questions.append(question)
        answers.append(answer)
        
print(len(questions))
print(len(answers))

questions_embed = model.encode(questions, show_progress_bar=True)
answers_embed = model.encode(answers, show_progress_bar=True)

print(questions_embed.shape)
print(answers_embed.shape)

48
48



(48, 768)
(48, 768)


In [21]:
feature_path = "../../SoR_Alberta.Shared.Data.and.Codebook.xlsx"#"../data/gr3/gr3_features.xlsx"
feature_names = ['G3.PPVT.Vocab.raw',
                 'G3.Elision.PA.raw',
                 'G3.Syn.GramCorrect.raw',
                 'G4.TOWRE.SWE.raw',
                 'G4.TOWRE.PDE.raw',
                 'G4.WordID.raw',
                 'G3.OL.Spell.Total',
                 'G3.OL.OrthoChoice.1.2.Total',
                 'G3.DigitSpan.raw',]


In [30]:
data = []
df2 = pd.read_excel(feature_path)

for i in df.index:
    is_available = True
    skills = []
    for name in feature_names:
        value = df2[name][i]
        if value < 0:
            is_available = False
            break
        skills.append(value)
    if not is_available:
        continue
    skills = np.asarray(skills)
    for index, q_range in enumerate(questions_ranges):
        entry = []
        entry.append(skills)
        entry.append(sub_tests_embed[index])
        for j in range(q_range[0],q_range[1]+1):
            detail = []
            detail.append(questions_embed[j-1])
            detail.append(answers_embed[j-1])
            if df[f"Gr{grade}.RC.Gates_"+"{:02d}".format(j)][i] == 1:
                detail.append(1)
            elif df[f"Gr{grade}.RC.Gates_"+"{:02d}".format(j)][i] in (2,0):
                detail.append(0)
            else:
                print('DNE')
            data.append(entry+detail)

print(len(data))
    

5808


In [33]:
with open("data.pkl",'wb') as fp:
    pickle.dump(data, fp)


In [34]:
with open("data.pkl",'rb') as fp:
    new_data = pickle.load(fp)

In [36]:
print(new_data[0])

[array([33, 26,  5, 71, 26, 61,  8, 21, 11]), array([-5.27166016e-02,  1.95111129e-02, -1.13468714e-01, -6.54139593e-02,
        8.90918896e-02,  1.04720742e-02,  3.28070402e-01,  4.23632897e-02,
       -2.70392708e-02, -1.43078631e-02,  4.35752571e-02, -1.70819715e-01,
       -4.45311852e-02, -2.40526989e-01, -8.33250284e-02, -1.22428844e-02,
       -4.37446237e-02,  8.61228704e-02,  1.10468715e-01,  5.68504073e-02,
        3.67352031e-02,  2.27153990e-02,  6.76281080e-02,  3.96065563e-02,
       -9.72608849e-02,  1.01404086e-01,  1.02220647e-01,  1.51116950e-02,
        7.29609728e-02, -6.33764789e-02,  1.76194549e-01,  4.58289236e-02,
       -7.66886547e-02, -1.19962409e-01,  4.42762719e-03,  4.90530953e-02,
        2.58376282e-02,  3.54838260e-02, -1.52256995e-01, -1.26583219e-01,
       -1.91097334e-01,  6.78996556e-03, -7.67501583e-03,  1.08349230e-03,
       -4.79228981e-02,  5.06950580e-02,  8.30517262e-02,  1.19973095e-02,
       -1.64606228e-01, -7.51331970e-02,  2.68952940e-